<a href="https://colab.research.google.com/github/julianox5/Exercicios-Programacao-Curso-Machine-Learning-do-GoogleDevelopers/blob/master/05_Conjunto_de_Validacao_e_Teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Conjunto de validação e de teste
Os exercícios anteriores da Colab avaliaram o modelo treinado em relação ao conjunto de treinamento, o que não fornece um sinal forte sobre a qualidade do seu modelo. Neste Colab, você experimentará conjuntos de validação e conjuntos de testes.

## Objetivos de aprendizado
Depois deste Colab, você saberá como fazer o seguinte:
* Dividir um conjunto de treinamento em um conjunto menor de treinamento e um conjunto de validação.
* Analise deltas entre os resultados do conjunto de treinamento e do conjunto de validação.
* Testar o modelo treinado com um conjunto de testes para determinar se seu modelo treinado está se ajustando demais.
* Detectar e corrigir um problema de treinamento comum.

## O conjunto de dados
Como no exercício anterior, este exercício usa o conjunto de dados da California Housing para prever o `median_house_value`. Como muitos conjuntos de dados "famosos", o California Housing Dataset na verdade consiste em dois conjuntos de dados separados, cada um vivendo em arquivos .csv separados:
* O conjunto de treinamento é `california_housing_train.csv`
* O conjunto de teste é `california_housing_test.csv`

Vamos criar o conjunto de validação dividindo o conjunto de treinamento baixado em duas partes:
* Um conjunto de treinamento menor
* Um conjunto de validação


## Usar a versão correta do TensorFlow


In [0]:
%tensorflow_version 2.x 

## Importar modúlos nescessários
Essa primeira célula de código importa os módulos necessários e define algumas opções de exibição.

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

##Carregar os conjuntos de dados da Internet
Carregando os arquivos .csv separados e criando os dois DataFrames do pandas a seguir:
* Conjunto de treinamento `df_treinamento`
* Conjunto de teste `df_test`


In [0]:
df_treinamento = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
df_teste       = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")


##Escalar os valores dos rótulos
Dimensionando o median_house_value. Veja o exercício anterior da Colab para obter detalhes.


In [0]:
df_treinamento["median_house_value"] /= 1000.0
df_teste["median_house_value"] /= 1000.0


Carregar as funções para construir e treinar um modelo





In [17]:
def contruir_modelo(tx_aprendizado):

  modelo = tf.keras.models.Sequential()
 
  modelo.add(tf.keras.layers.Dense(units=1,
                                   input_shape=(1,)))
  
  modelo.compile(optmizer=tf.keras.optimizers.RMSprop(lr=tx_aprendizado,
                                                      loss="mean_squared_error",
                                                      metrics=[tf.keras.metrics.RootMeanSquaredError()]))
  return modelo

def treinar_modelo(modelo, df, recurso, rotulo,epocas, tam_lote=None, div_validacao=0.1):
  
  history = modelo.fit(x=recurso, y = rotulo,
                       epochs = epocas, batch = tam_lote, 
                       validation_split = div_validacao)
  
  peso_treinado = modelo.get_weights()[0]
  vies_treinado = modelo.get_weights()[1]

  epochs = history.epoch

  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history 

print("Ok!")

Ok!


## Definir Funções de plotagem
A função `plot_curva_perda` plota perda versus época para o conjunto de treinamento e o conjunto de validação


In [18]:
def plot_curva_perda(epochs, set_treinamento, set_validacao):

  plt.figure()
  plt.xlabel("Época")
  plt.ylabel("Erro médio quadrático da raiz")

  plt.plot(epochs[1:], set_treinamento, label="Perda Treinamento")
  plt.plot(epochs[1:], set_validacao, label="Perda validação")
  plt.legend()
  # Não vamos traçar a primeira época, já qua perda na primeira época
  # geralmente é substancialmente maior que a perda para outras épocas.
  lista_fundida = set_treinamento[1:] + set_validacao[1:]

  maior_perda = max(lista_fundida)
  menor_perda = min(lista_fundida)
  delta = maior_perda - menor_perda
  print(delta)

  topo_eixoY = maior_perda + (delta * 0.05)
  fundo_eixoX = menor_perda - (delta * 0.05)

  plt.ylim([fundo_eixoX, topo_eixoY])
  plt.show()

print("Definida função para plotar gráfico da curva de perda!")


Definida função para plotar gráfico da curva de perda!


## Tarefa 1: Experiência com a divisão de validação
Na célula de código a seguir, você verá uma variável denominada `validacao_div`, que inicializamos em 0,2. A validation_splitvariável especifica a proporção do conjunto de treinamento original que servirá como o conjunto de validação. O conjunto de treinamento original contém 17.000 exemplos. Portanto, um `validacao_div` 0,2 significa que:
* 17.000 * 0,2 ~ = 3.400 exemplos se tornarão o conjunto de validação.
* 17.000 * 0,8 ~ = 13.600 exemplos se tornarão o novo conjunto de treinamento.

O código a seguir cria um modelo, o treina no conjunto de treinamento e avalia o modelo criado em ambos:
* Conjunto de treinamento
* Conjunto de validacao

Se os dados no conjunto de treinamento forem semelhantes aos dados no conjunto de validação, as duas curvas de perda e os valores de perda final deverão ser quase idênticos. No entanto, as curvas de perda e os valores de perda final não são quase idênticos. Hmm, isso é estranho.

Experimente com dois ou três valores diferentes de `validacao_div`. Valores diferentes de `validacao_div`correção do problema?

In [15]:
# Difinindo os hiperparâmetros
tx_aprendizado = 0.08
epochs = 30 
tam_lote = 100

#dividindo o conjunto de treinamento original em um conjunto reduzido e um 
#conjunto de treinamento
validacao_div = 0.2

#identifica o recurso e o rótulo 
meu_recurso = "median_income" # renda media em um quarteirao específico
meu_rotulo  = "median_house_value" # o valor medio de uma casa em uma cidade expecífica por bloco
# Vamos criar um modelo que prevê o valor da casa com base na renda media do bairro

# descartar qualquer versão pré-existente do modelo.
meu_modelo = None

# invocar as funções para criar e treinar o modelo
meu_modelo = contruir_modelo(tx_aprendizado)

epochs, rmse, history = treinar_modelo(meu_modelo, df_treinamento, 
                                       meu_recurso, meu_rotulo, epochs, tam_lote,
                                       validacao_div)
plot_curva_perda(epochs, history["root_mean_squared_error"], 
                    history["val_root_mean_squared_error"])

TypeError: ignored